## Lines data

In [ ]:
# import modules
import geopandas as gpd
import folium
import json
import branca

### Loading LineString data

Let us work now with *line-shaped* geospatial data using a dataset representing speed limits on various road segments in the Netherlands. 

`geopandas` uses `shapely` *LineString* objects to represent this type of `geometry`; `LineString` objects consists of one or more connected linear splines between points. 

In [ ]:
# load only a part of dataset to avoid problems with computational resources
max_speed_df = gpd.read_file('Data/linestring/max_snelheden.shp', rows=2000) 

In [ ]:
max_speed_df.head()

This time the "OMSCHR" column represent the speed limits on the road segment, geometrically defined by the linestrings rightmost "geometry" column.

The first linestring is defined by a sequence of 3 points

In [ ]:
print(max_speed_df.geometry[0])

In [ ]:
max_speed_df.geometry[0]

Remember that we can always use `geopandas` functionalities to extract interesting information from our `geometry` column, such as the length of each road segment:

In [ ]:
max_speed_df.length

### LineString visualization

Let's create a base map centered with respect to the road coordinates in our GeoDataFrame. We do this by first computing the average coordinate of each `LineString`. Since `folium` default CRS is "EPSG:4326", we need to transform our geometries accordingly, using the `to_crs` method.

In [ ]:
max_speed_df.geometry.head()

In [ ]:
transformed_geometries = max_speed_df.geometry.to_crs('EPSG:4326')
transformed_geometries.head()

The `bounds` attribute returns a (minx, miny, maxx, maxy) tuple (float values) that bounds each geometric object.

In [ ]:
transformed_geometries.bounds.head()

We can use this information to find the central point of each bounding box for each road, and compute the overall latitude and longitude.

In [ ]:
avg_latitude = ((transformed_geometries.bounds.maxy+transformed_geometries.bounds.miny)/2).mean()
avg_longitude = ((transformed_geometries.bounds.maxx+transformed_geometries.bounds.minx)/2).mean()

In [ ]:
# create
nl_map = folium.Map(
    location=[avg_latitude,avg_longitude],
    zoom_start=10
)

# display
nl_map

Similar to what seen for the Point data, we can draw all the `LineString` objects on `GeoJson` object sitting on top of the base map.

In [ ]:
# create
gjson = folium.features.GeoJson(
    max_speed_df).add_to(nl_map)

# display
nl_map

We can make the plot more informative by changing the line colors according to speed limits.

First, we create a color map using the [branca](https://github.com/python-visualization/branca) Python library; `branca` is a *spinoff* from `folium` hosting non-map-specific features such as colormaps. 

In this case, we use a linear colormap going from Red (Rd) to Blue (Bu) through Yellow (Yl). We scale the colors between the min and max reported speeds by using the `vmin` and `vmax` arguments.

In [ ]:
colors = branca.colormap.linear.RdYlBu_11.scale(vmin=max_speed_df.OMSCHR.min(), vmax=max_speed_df.OMSCHR.max())
colors

To customise the visualisation of each `LineString`, we define and apply a `style_function` to all rows of the GeoDataFrame (similar to what happens when using the `pandas.DataFrame.apply` construct). 

The main difference is that the rows of the GeoDataFrame, as passed by `folium` to our `style_function`, are now encoded in the underlying JSON format. Their data is now stored into a `properties` dictionary.

The `style_function` returns a dictionary of attributes computed for each row, to be applied to the relative `geometry` visualized using `folium`. In particular, the function below sets the color according to the speed limit of each row (e.g., road) and the colorscale we defined before. The function and also fix the values for the opacity of each plotted segment and the weight (e.g., thickness).

In [ ]:
def style_function(row):
    return {
        "color": colors(row['properties']['OMSCHR']),
        "fillOpacity": 0.5,
        "weight": 5,        
    }

Now, we can create another overlay with the customised style settings

In [ ]:
# create
gjson = folium.features.GeoJson(
    max_speed_df,
    style_function=style_function,
).add_to(nl_map)

# display
nl_map

Finally, we can show extra information using "pop-up" windows.

In [ ]:
# create
folium.features.GeoJsonPopup(
    fields=['WVK_ID', 'OMSCHR'],
    aliases=['Segment_ID', 'Maximum speed'],
    labels=True
).add_to(gjson)

# display
nl_map

### Exercise

You are given a geospatial dataset for the roads in Vietnam

In [ ]:
df_vietroads = gpd.read_file('./Data/case_study/vietnam_roadnet.geojson')

In [ ]:
df_vietroads.plot();

The dataset contains geometrical information on the location of the road along with contextual data such as information on the road type ("rtt" column)

In [ ]:
df_vietroads.head()

In [ ]:
df_vietroads.rtt.value_counts().sort_index()

Your task is to use `folium` as shown earlier in the notebook to display a map for the roads of Vietnam where:

1. The map is centered on Hanoi, e.g. `latitude = 21.03 N`, `longitude = 105.8 E`;
2. Each road type "rtt" is displayed with a different color. The following colors (in hexadecimal format) are assigned to each road type 
    - 14: `#E74C3C`
    - 15: `#28B463`
    - 16: `#85C1E9`
    - 999:`#2C3E50`
3. Colors are assigned with `folium.features.GeoJson` using a `style_function`;
4. The map has pop-up windows enabled showing information on "rtt".

> Hints: Remember to transform the crs of the loaded dataset to the one used by `folium`. Instead of using a `branca` colormap, you can simply use a dictionary mapping different road types to the selected colors.

#### Solution

In [ ]:
""" Your code here"""